In [2]:
import numpy as np
import pandas as pd
from patsy import dmatrices
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
import matplotlib.pyplot as plt

In [3]:
data = pd.read_csv("HR_comma_sep.csv")
data.head(10)

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low
5,0.41,0.50,2,153,3,0,1,0,sales,low
6,0.10,0.77,6,247,4,0,1,0,sales,low
7,0.92,0.85,5,259,5,0,1,0,sales,low
8,0.89,1.00,5,224,5,0,1,0,sales,low
9,0.42,0.53,2,142,3,0,1,0,sales,low


In [4]:
y, X = dmatrices('left~satisfaction_level+last_evaluation+number_project+average_montly_hours+time_spend_company+Work_accident+promotion_last_5years+C(sales)+C(salary)', data, return_type='dataframe')
X = np.asmatrix(X)
y = np.ravel(y)

In [5]:
# Normalization by each column
for i in range(1, X.shape[1]):
    xmin = X[:,i].min()
    xmax = X[:,i].max()
    X[:, i] = (X[:, i] - xmin) / (xmax - xmin)

In [6]:
X

matrix([[1.   , 0.   , 0.   , ..., 0.125, 0.   , 0.   ],
        [1.   , 0.   , 0.   , ..., 0.5  , 0.   , 0.   ],
        [1.   , 0.   , 0.   , ..., 0.25 , 0.   , 0.   ],
        ...,
        [1.   , 0.   , 0.   , ..., 0.125, 0.   , 0.   ],
        [1.   , 0.   , 0.   , ..., 0.25 , 0.   , 0.   ],
        [1.   , 0.   , 0.   , ..., 0.125, 0.   , 0.   ]])

### Logistic Regression Class

In [30]:
class MyLogisticReg():
    
    def __init__(self, max_iters=200, lr=2, eps=1e-5, thres=0.5, random_state=0):
        self.max_iters = max_iters
        self.lr = lr
        self.eps = eps
        self.thres = thres
        self.random_state = random_state
        self.beta = None
        
    def _randParameters(self, X, y):
        _, n_features = X.shape
        beta = np.random.randn(n_features)
        return beta
    
    def fit(self, X, y):
        np.random.seed(self.random_state)
        losses = []
        error_rates = []
        # init beta
        beta = self._randParameters(X, y)
        
        for T in range(self.max_iters):
            # p(Y=1|X) = 1 / [1 + exp(-X*beta)]
            probs = np.array(1. / (1 + np.exp(- np.matmul(X, beta)))).ravel()
            # cross entropy loss: -1/N* sum {y_i*log(p_i) + (1-y_i)*log(1-p_i)}
            loss = 0
            for i in range(len(y)):
                if y[i] == 1:
                    loss += np.log(probs[i])
                else:
                    loss += np.log(1 - probs[i])
            loss = - loss / len(y)
            losses.append(loss)
            
            probs_y = list(zip(probs, y))
            error_count = 0
            for p_i, y_i in probs_y:
                if ((y_i == 1 and p_i < self.thres ) or (y_i == 0 and p_i >= self.thres )):
                    error_count += 1
            error = error_count / len(y)
            error_rates.append(error)
            
            if T % 10 == 0:
                print("T=" + str(T)+ " loss=" + str(loss) + " error rate=", str(error))
            
            # update beta by GD
            deriv = np.zeros(X.shape[1])
            for i in range(len(y)):
                deriv += np.asarray(X[i,:]).ravel() * (probs[i] - y[i])
            deriv /= len(y)
            beta_new = beta - self.lr * deriv
            
            # check 
            if (np.sqrt(sum((beta_new - beta)**2)) < self.eps):
                break   
            else:
                beta = beta_new
        self.beta = beta
        
    def predict_prob(self, X):
        prob = np.array(1. / (1 + np.exp(- np.matmul(X, self.beta)))).ravel()
        return prob
    
    def predict_class(self, X):
        prob = self.predict_prob(X)
        prediction = 1 * (prob > self.thres)
        return prediction


In [33]:
LR_model = MyLogisticReg(lr=2.5, eps=1e-6, random_state=941215)
LR_model.fit(X, y)

T=0 loss=0.8220714960178823 error rate= 0.46869791319421295
T=10 loss=0.566680786765763 error rate= 0.24901660110674045
T=20 loss=0.509399643906681 error rate= 0.22594839655977064
T=30 loss=0.48095241090885 error rate= 0.2125475031668778
T=40 loss=0.4648214299441914 error rate= 0.20948063204213616
T=50 loss=0.4548199670903523 error rate= 0.21168077871858124
T=60 loss=0.4482525204536383 error rate= 0.2142809520634709
T=70 loss=0.44375655569040934 error rate= 0.21688112540836055
T=80 loss=0.44057190889648196 error rate= 0.21901460097339823
T=90 loss=0.43824837855197524 error rate= 0.21921461430762051
T=100 loss=0.43650824292053436 error rate= 0.2194146276418428
T=110 loss=0.43517455377844083 error rate= 0.21694779651976798
T=120 loss=0.4341313433915386 error rate= 0.21494766317754516
T=130 loss=0.43330062206484005 error rate= 0.21368091206080406
T=140 loss=0.43262866549049395 error rate= 0.2118141209413961
T=150 loss=0.43207762521489834 error rate= 0.20994732982198813
T=160 loss=0.431620

In [35]:
from sklearn.metrics import confusion_matrix
prediction = LR_model.predict_class(X)
print(confusion_matrix(y, prediction))

[[10661   767]
 [ 2322  1249]]
